# Playing 2048 with Minimax and Selenium

[Here](https://towardsdatascience.com/tagged/playing-2048-with-minimax) are a few articles that explain this project.

In [1]:
from copy import deepcopy
from typing import Tuple, List
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from sys import maxsize as MAX_INT
import time

### Here are the move direction codes:
- 0 = Up
- 1 = Down
- 2 = Left
- 3 = Right

In [2]:
class Grid:
    
    def __init__(self, matrix):
        self.setMatrix(matrix)
    
    def __eq__(self, other) -> bool:
        for i in range(4):
            for j in range(4):
                if self.matrix[i][j] != other.matrix[i][j]:
                    return False
        return True
    
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)
    
    def placeTile(self, row: int, col: int, tile: int):
        self.matrix[row-1][col-1] = tile
    
    def utility(self) -> int:
        count = 0
        sum = 0
        for i in range(4):
            for j in range(4):
                sum += self.matrix[i][j]
                if self.matrix[i][j] != 0:
                    count += 1
        return int(sum/count)
    
    def canMoveUp(self) -> bool:
        for j in range(4):
            k = -1
            for i in range(3, -1, -1):
                if self.matrix[i][j] > 0:
                    k = i
                    break
            if k > -1:
                for i in range(k, 0, -1):
                    if self.matrix[i-1][j] == 0 or self.matrix[i][j] == self.matrix[i-1][j]:
                        return True
        return False

    def canMoveDown(self) -> bool:
        for j in range(4):
            k = -1
            for i in range(4):
                if self.matrix[i][j] > 0:
                    k = i
                    break
            if k > -1:
                for i in range(k, 3):
                    if self.matrix[i+1][j] == 0 or self.matrix[i][j] == self.matrix[i+1][j]:
                        return True
        return False

    def canMoveLeft(self) -> bool:
        for i in range(4):
            k = -1
            for j in range(3, -1, -1):
                if self.matrix[i][j] > 0:
                    k = j
                    break
            if k > -1:
                for j in range(k, 0, -1):
                    if self.matrix[i][j-1] == 0 or self.matrix[i][j] == self.matrix[i][j-1]:
                        return True
        return False

    def canMoveRight(self) -> bool:
        for i in range(4):
            k = -1
            for j in range(4):
                if self.matrix[i][j] > 0:
                    k = j
                    break
            if k > -1:
                for j in range(k, 3):
                    if self.matrix[i][j+1] == 0 or self.matrix[i][j] == self.matrix[i][j+1]:
                        return True
        return False
    
    def getAvailableMovesForMax(self) -> List[int]:
        moves = []

        if self.canMoveUp():
            moves.append(0)
        if self.canMoveDown():
            moves.append(1)
        if self.canMoveLeft():
            moves.append(2)
        if self.canMoveRight():
            moves.append(3)
        
        return moves
    
    def getAvailableMovesForMin(self) -> List[Tuple[int]]:
        places = []
        for i in range(4):
            for j in range(4):
                if self.matrix[i][j] == 0:
                    places.append((i+1, j+1, 2))
                    places.append((i+1, j+1, 4))
        return places
    
    def getChildren(self, who: str) -> List:
        if who == "max":
            return self.getAvailableMovesForMax()
        elif who == "min":
            return self.getAvailableMovesForMin()
    
    def isTerminal(self, who: str) -> bool:
        if who == "max":
            if self.canMoveUp():
                return False
            if self.canMoveDown():
                return False
            if self.canMoveLeft():
                return False
            if self.canMoveRight():
                return False
            return True
        elif who == "min":
            for i in range(4):
                for j in range(4):
                    if self.matrix[i][j] == 0:
                        return False
            return True
    
    def isGameOver(self) -> bool:
        return self.isTerminal(who="max")
    
    def up(self):
        for j in range(4):
            w = 0
            k = 0
            for i in range(4):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[w][j] = 2*k
                    w += 1
                    k = 0
                else:
                    self.matrix[w][j] = k
                    w += 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[w][j] = k
                w += 1
            for i in range(w, 4):
                self.matrix[i][j] = 0
    
    def down(self):
        for j in range(4):
            w = 3
            k = 0
            for i in range(3, -1, -1):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[w][j] = 2*k
                    w -= 1
                    k = 0
                else:
                    self.matrix[w][j] = k
                    w -= 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[w][j] = k
                w -= 1
            for i in range(w+1):
                self.matrix[i][j] = 0
    
    def left(self):
        for i in range(4):
            w = 0
            k = 0
            for j in range(4):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[i][w] = 2*k
                    w += 1
                    k = 0
                else:
                    self.matrix[i][w] = k
                    w += 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[i][w] = k
                w += 1
            for j in range(w, 4):
                self.matrix[i][j] = 0
    
    def right(self):
        for i in range(4):
            w = 3
            k = 0
            for j in range(3, -1, -1):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[i][w] = 2*k
                    w -= 1
                    k = 0
                else:
                    self.matrix[i][w] = k
                    w -= 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[i][w] = k
                w -= 1
            for j in range(w+1):
                self.matrix[i][j] = 0
    
    def move(self, mv: int) -> None:
        if mv == 0:
            self.up()
        elif mv == 1:
            self.down()
        elif mv == 2:
            self.left()
        else:
            self.right()
    
    def getMoveTo(self, child: 'Grid') -> int:
        if self.canMoveUp():
            g = Grid(matrix=self.getMatrix())
            g.up()
            if g == child:
                return 0
        if self.canMoveDown():
            g = Grid(matrix=self.getMatrix())
            g.down()
            if g == child:
                return 1
        if self.canMoveLeft():
            g = Grid(matrix=self.getMatrix())
            g.left()
            if g == child:
                return 2
        return 3

In [3]:
class GameDriver:    
    def __init__(self):
        self.url = 'https://hczhcz.github.io/2048/20ez/'
        self.driver = webdriver.Chrome()
        self.driver.get(self.url)
        self.body = self.driver.find_element_by_tag_name('body')
        self.moves = {
            0: Keys.ARROW_UP,
            1: Keys.ARROW_DOWN,
            2: Keys.ARROW_LEFT,
            3: Keys.ARROW_RIGHT
        }
    
    def getGrid(self) -> Grid:
        matrix = [[0 for i in range(4)] for j in range(4)]
        tiles = self.driver.find_elements_by_class_name('tile')
        
        for tile in tiles:
            cls = tile.get_attribute('class')
            col, row = cls.split('tile-position-')[1].split(' ')[0].split('-')
            col, row = int(col)-1, int(row)-1
            num = int(cls.split('tile tile-')[1].split(' ')[0])
            if num > matrix[row][col]:
                matrix[row][col] = num
        
        return Grid(matrix)
    
    def move(self, moveCode):
        self.body.send_keys(self.moves[moveCode])
        time.sleep(0.1)

In [4]:
def maximize(state: Grid, a: int, b: int, d: int) -> Tuple[Grid, int]:
    (maxChild, maxUtility) = (None, -1)

    if d == 0 or state.isTerminal(who="max"):
        return (None, state.utility())
    
    d -= 1
    
    for child in state.getChildren(who = "max"):
        grid = Grid(matrix=state.getMatrix())
        grid.move(child)
        (_, utility) = minimize(grid, a, b, d)
        if utility > maxUtility:
            (maxChild, maxUtility) = (grid, utility)
        if maxUtility >= b:
            break
        if maxUtility > a:
            a = maxUtility

    return (maxChild, maxUtility)

def minimize(state: Grid, a: int, b: int, d: int) -> Tuple[Grid, int]:
    (minChild, minUtility) = (None, MAX_INT)

    if d == 0 or state.isTerminal(who="min"):
        return (None, state.utility())

    d -= 1
    
    for child in state.getChildren(who = "min"):
        grid = Grid(matrix=state.getMatrix())
        grid.placeTile(child[0], child[1], child[2])
        (_, utility) = maximize(grid, a, b, d)
        if utility < minUtility:
            (minChild, minUtility) = (grid, utility)
        if minUtility <= a:
            break
        if minUtility < b:
            b = minUtility

    return (minChild, minUtility)

def getBestMove(grid: Grid, depth: int = 5):
    (child, _) = maximize(Grid(matrix=grid.getMatrix()), -1, MAX_INT, depth)
    return grid.getMoveTo(child)

In [5]:
gameDriver = GameDriver()

In [6]:
moves_str = ['UP', 'DOWN', 'LEFT', 'RIGHT']
moves_count = 1
while True:
    grid = gameDriver.getGrid()
    if grid.isGameOver():
        print("Unfortunately, I lost the game.")
        break
    moveCode = getBestMove(grid, 5)
    print(f'Move #{moves_count}: {moves_str[moveCode]}')
    gameDriver.move(moveCode)
    moves_count += 1

Move #1: UP
Move #2: LEFT
Move #3: DOWN
Move #4: LEFT
Move #5: UP
Move #6: DOWN
Move #7: RIGHT
Move #8: UP
Move #9: UP
Move #10: UP
Move #11: DOWN
Move #12: UP
Move #13: UP
Move #14: LEFT
Move #15: DOWN
Move #16: LEFT
Move #17: UP
Move #18: LEFT
Move #19: UP
Move #20: DOWN
Move #21: UP
Move #22: RIGHT
Move #23: UP
Move #24: UP
Move #25: LEFT
Move #26: LEFT
Move #27: UP
Move #28: UP
Move #29: LEFT
Move #30: LEFT
Move #31: UP
Move #32: UP
Move #33: UP
Move #34: UP
Move #35: UP
Move #36: DOWN
Move #37: LEFT
Move #38: DOWN
Move #39: RIGHT
Move #40: DOWN
Move #41: DOWN
Move #42: DOWN
Move #43: DOWN
Move #44: DOWN
Move #45: DOWN
Move #46: DOWN
Move #47: RIGHT
Move #48: UP
Move #49: RIGHT
Move #50: RIGHT
Move #51: UP
Move #52: UP
Move #53: UP
Move #54: DOWN
Move #55: UP
Move #56: DOWN
Move #57: UP
Move #58: LEFT
Move #59: UP
Move #60: RIGHT
Move #61: UP
Move #62: UP
Move #63: DOWN
Move #64: UP
Move #65: LEFT
Move #66: UP
Move #67: DOWN
Move #68: DOWN
Move #69: LEFT
Move #70: UP
Move #71: UP
M

Move #546: UP
Move #547: LEFT
Move #548: DOWN
Move #549: UP
Move #550: LEFT
Move #551: UP
Move #552: LEFT
Move #553: RIGHT
Move #554: UP
Move #555: UP
Move #556: DOWN
Move #557: UP
Move #558: UP
Move #559: LEFT
Move #560: RIGHT
Move #561: UP
Move #562: DOWN
Move #563: DOWN
Move #564: LEFT
Move #565: DOWN
Move #566: LEFT
Move #567: UP
Move #568: UP
Move #569: UP
Move #570: DOWN
Move #571: RIGHT
Move #572: UP
Move #573: UP
Move #574: DOWN
Move #575: UP
Move #576: UP
Move #577: DOWN
Move #578: UP
Move #579: DOWN
Move #580: UP
Move #581: DOWN
Move #582: UP
Move #583: UP
Move #584: LEFT
Move #585: DOWN
Move #586: UP
Move #587: RIGHT
Move #588: UP
Move #589: UP
Move #590: UP
Move #591: LEFT
Move #592: LEFT
Unfortunately, I lost the game.
